#### Multi-message eavesdropping experiment using the Vigenère cipher

In [1]:
# Preparations
ES = VigenereCryptosystem(AlphabeticStrings(), 10)

# Step 1: Adversary chooses two message vectors
M0 = ['ABCDEFGHIJ', 'ABCDEFGHIJ']
M1 = ['SECRETMESS', 'CRYPTOGRAP']
M0_ = []
for m in M0:
    M0_.append(ES.encoding(m))
M1_ = []
for m in M1:
    M1_.append(ES.encoding(m))

In [6]:
# Step 2: Random key and random bit are generated
k = ES.random_key() # Gen algorithm
b = randint(0, 1) # generation of a random bit b
C_ = []
if b == 0:
    for m in M0_:
        C_.append(ES.enciphering(k, m))
else:
    for m in M1_:
        C_.append(ES.enciphering(k, m))
print('Challenge ciphertext: ' + str(C_))

Challenge ciphertext: [BONJZZTETQ, BONJZZTETQ]


#### RC4 encryption scheme WITHOUT randomisation (probabilistic encryption)

In [7]:
def rc4(key, count):
    ks = []
    l = len(key)
    j = 0
    s = [si for si in range(256)]
    for i in range(256):
        j = (j + s[i] + key[i % l]) % 256
        s[i], s[j] = s[j], s[i]
    i = 0
    j = 0
    for n in range(count):
        i = (i + 1) % 256
        j = (j + s[i]) % 256
        s[i], s[j] = s[j], s[i]
        ks.append(s[(s[i] + s[j]) % 256])
    return ks

def Gen(n):
    l = round(n/8) # n: length of key in bits, l: number of bytes in key
    k = []
    for i in range(l):
        k.append(randint(0, 256))
    return k

def Enc(k, m):
    ks = rc4(k, len(m))
    c = []
    for i in range(len(m)):
        c.append(ks[i]^^ord(m[i]))
    return c

def Dec(k, c):
    ks = rc4(k, len(c))
    m = ''
    for i in range(len(c)):
        m += chr(ks[i]^^c[i])
    return m

#### Encryption example

In [8]:
k = Gen(128) # 128 bit key
m = 'crypto @ fh kiel'
c = Enc(k, m)
c

[157, 153, 130, 126, 8, 94, 5, 68, 24, 252, 217, 248, 176, 116, 142, 190]

#### Decryption

In [9]:
Dec(k, c)

'crypto @ fh kiel'

#### Multi-message eavesdropping experiment (cf. Slide 22)
Let $\Pi=(\operatorname{Gen},\operatorname{Enc},\operatorname{Dec})$ be a symmetric-key encryption scheme with security parameter $n\in\mathbb{N}$, let $\mathcal{A}$ be an eavesdropping adversary.
1. $\mathcal{A}$ gets $n$ as input and outputs a pair of vectors $\vec{M}_0=(m_0^1,\dots,m_0^t)$ und $\vec{M}_1=(m_1^1,\dots,m_1^t)$, each containing messages with $|m_0^i|=|m_1^i|$ for all $i$.
2. A key $k$ is generated by running $\operatorname{Gen}(n)$ and a random bit $b\leftarrow\lbrace0,1\rbrace$ in produced. For all $i$, the ciphertext $c^i\leftarrow\operatorname{Enc}_k(m_b^i)$ is computed and $\mathcal{A}$ gets the vector of ciphertexts $\vec{C}=(c^1,\dots,c^t)$.
3. $\mathcal{A}$ outputs a bit $b'$.
4. The output of the experiment is defined to be $1$ if $b'=b$, and $0$ otherwise. For
			$$ \operatorname{SymmK}_{\mathcal{A},\Pi}^{\operatorname{mult}}(n)=1, $$
			we say that $\mathcal{A}$ succeeds.

In [10]:
# Step 1
M0 = ['crypto', 'crypto', 'crypto']
M1 = ['crypto', 'fhkiel', 'abcdef']

In [12]:
# Step 2
k = Gen(128)
b = randint(0, 1)
C = []
if b == 0:
    for m in M0:
        C.append(Enc(k, m))
else:
    for m in M1:
        C.append(Enc(k, m))
print(C)

[[206, 253, 23, 6, 185, 25], [203, 231, 5, 31, 168, 26], [204, 237, 13, 18, 168, 16]]


#### RC4 encryption scheme WITH randomisation (probabilistic encryption)

In [13]:
# We don't touch the RC4 algorithm and we also don't touch the key generation.

def Enc(k, m):
    iv = [randint(0, 255) for i in range(3)]
    k_session = iv + k
    ks = rc4(k_session, len(m))
    c = iv # first three bytes of ciphertext are bytes of IV
    for i in range(len(m)):
        c.append(ks[i]^^ord(m[i]))
    return c

def Dec(k, c):
    iv = c[0:3] # get first three bytes as IV
    k_session = iv + k
    ks = rc4(k_session, len(c)-3)
    m = ''
    for i in range(len(c)-3):
        m += chr(ks[i]^^c[i+3])
    return m

#### Encryption example

In [25]:
k = Gen(128)
m = 'crypto @ fh kiel'
c = Enc(k, m)
print(c)

[67, 195, 138, 65, 39, 106, 72, 73, 238, 60, 70, 128, 86, 20, 19, 85, 163, 207, 185]


#### Decryption

In [26]:
Dec(k, c)

'crypto @ fh kiel'

#### Multi-message eavesdropping experiment (cf. Slide 22)

In [27]:
# Step 1
M0 = ['crypto', 'crypto', 'crypto']
M1 = ['crypto', 'fhkiel', 'abcdef']

In [30]:
# Step 2
k = Gen(128)
b = randint(0, 1)
C = []
if b == 0:
    for m in M0:
        C.append(Enc(k, m))
else:
    for m in M1:
        C.append(Enc(k, m))
print(C)

[[124, 45, 179, 32, 18, 105, 101, 251, 129], [100, 57, 38, 92, 121, 198, 94, 10, 48], [14, 218, 214, 129, 77, 16, 82, 44, 78]]


In [31]:
b

0

In [33]:
N((253/256)^252)

0.0512743398647259